In [1]:
from pathlib import Path
from os.path import expanduser
from os import path
import numpy as np
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torchvision import transforms, datasets
from torch.optim import lr_scheduler
import os



In [2]:
root_folder = os.path.dirname(os.getcwd())
path_to_train_data = path.join(root_folder,'data/train_data_re_id.npy');
path_to_labels = path.join(root_folder,'data/train_labels_re_id.npy');
#path_to_mean_std = path.join(root_folder,'data/train_data/synthesized/mean_std.npy');

sys.path.append(root_folder)
from custom_resnet import CustomResnet as cnn

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 15000;
nb_of_epochs = 75;
logging_interval = 30
dataset_divider = 0.2

In [4]:
data = np.load(path_to_train_data);
labels = np.load(path_to_labels);
torch_data = torch.tensor(data).float()
torch_labels = torch.tensor(labels).int()
nb_of_classes = torch_labels.unique().size()[0]

In [5]:
print(nb_of_classes)

499


In [6]:
net = cnn.ft_net( nb_of_classes, 0);
net.to(device)

ft_net(
  (model): ResNet(
    (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
        (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
        (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
        (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU

In [7]:
if (len(torch_labels.size()) == 2):
    torch_labels = torch.squeeze(torch_labels)

In [8]:
# vertical flip
train_data_original, test_data_original = cnn.GenerateTrainAndTestDataset(torch_data, torch_labels, 0.2)
train_data_flipped, test_data_flippedl = cnn.GenerateTrainAndTestDataset(torch_data * -1, torch_labels, 0.2)



In [9]:
train_data = torch.utils.data.ConcatDataset((train_data_original, train_data_flipped));
test_data = torch.utils.data.ConcatDataset((test_data_original, test_data_flippedl));


In [10]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=8)





In [11]:
print(torch_data.size())
print(torch_labels.size())

torch.Size([2500489, 1, 72])
torch.Size([2500489])


In [12]:
print(torch_data[0,:])

tensor([[-5.2435e-01, -3.7025e-01, -1.3434e-01, -2.4474e-01, -3.5353e-01,
         -3.3476e-01, -2.0875e-01, -4.7216e-01, -7.2495e-01, -1.2463e+00,
         -1.5984e+00, -1.4817e+00, -1.7369e+00, -1.8890e+00, -2.1955e+00,
         -2.2463e+00, -2.0012e+00, -1.5047e+00, -1.1344e+00, -1.1903e+00,
         -1.4721e+00, -1.3439e+00, -9.3072e-01,  6.1741e-02,  7.0020e-01,
          1.2683e+00,  1.9363e+00,  2.1746e+00,  1.9996e+00,  1.6970e+00,
          1.5125e+00,  7.9792e-01, -6.8663e-02, -2.5527e+00, -7.1471e+00,
         -1.3885e+01, -1.8149e+01, -1.5931e+01, -9.2331e+00, -2.6818e+00,
          2.2164e+00,  5.5776e+00,  7.8141e+00,  9.1098e+00,  9.7197e+00,
          9.6600e+00,  8.8542e+00,  7.7477e+00,  6.6816e+00,  5.5360e+00,
          4.2213e+00,  3.1576e+00,  2.3921e+00,  1.9880e+00,  1.7718e+00,
          1.7307e+00,  1.8030e+00,  1.3844e+00,  5.8021e-01, -1.8272e-01,
         -9.4894e-01, -1.1664e+00, -1.1201e+00, -8.6691e-01, -3.7466e-01,
         -1.2474e-02,  3.5129e-01,  3.

In [13]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
exp_lr_scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[5, 25, 45, 65], gamma=0.1)
for i in range(nb_of_epochs):  # loop over the dataset multiple times
  loss = cnn.Train(net, device, train_loader, optimizer, criterion, exp_lr_scheduler, i, logging_interval)
  model_name = 'models/re_id/resnet18_num_classes_' + str(nb_of_classes) + "_epoch_" + str(i) + '.pt';
  path_to_model = path.join(root_folder, model_name)
  torch.save({'epoch': i,'model_state_dict': net.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'loss': loss,}, path_to_model)
  cnn.Test(net, device, criterion, train_loader)
  cnn.Test(net, device, criterion, test_loader)

Train Epoch: 0 [0/4000982 (0%)]	Loss: 6.21308660507202148438
Train Epoch: 0 [450000/4000982 (11%)]	Loss: 4.40636825561523437500
Train Epoch: 0 [900000/4000982 (22%)]	Loss: 2.69158697128295898438
Train Epoch: 0 [1350000/4000982 (34%)]	Loss: 1.82334101200103759766
Train Epoch: 0 [1800000/4000982 (45%)]	Loss: 1.44292294979095458984
Train Epoch: 0 [2250000/4000982 (56%)]	Loss: 1.27140760421752929688
Train Epoch: 0 [2700000/4000982 (67%)]	Loss: 1.10747385025024414062
Train Epoch: 0 [3150000/4000982 (79%)]	Loss: 1.07857441902160644531
Train Epoch: 0 [3600000/4000982 (90%)]	Loss: 0.95776104927062988281

Test set: Average loss: 0.00006167180981719866, Accuracy: 2644361/4000982 (66.09279921779203448295%)


Test set: Average loss: 0.00006202644726727158, Accuracy: 659874/999996 (65.98766395065580070423%)

Train Epoch: 1 [0/4000982 (0%)]	Loss: 0.90780311822891235352
Train Epoch: 1 [450000/4000982 (11%)]	Loss: 0.89247435331344604492
Train Epoch: 1 [900000/4000982 (22%)]	Loss: 0.8648160696029663085

Train Epoch: 10 [900000/4000982 (22%)]	Loss: 0.49073308706283569336
Train Epoch: 10 [1350000/4000982 (34%)]	Loss: 0.48457667231559753418
Train Epoch: 10 [1800000/4000982 (45%)]	Loss: 0.47329166531562805176
Train Epoch: 10 [2250000/4000982 (56%)]	Loss: 0.47743666172027587891
Train Epoch: 10 [2700000/4000982 (67%)]	Loss: 0.48319813609123229980
Train Epoch: 10 [3150000/4000982 (79%)]	Loss: 0.47607895731925964355
Train Epoch: 10 [3600000/4000982 (90%)]	Loss: 0.47026523947715759277

Test set: Average loss: 0.00002939320984296501, Accuracy: 3321634/4000982 (83.02046847498938575427%)


Test set: Average loss: 0.00003094838757533580, Accuracy: 820484/999996 (82.04872819491278335136%)

Train Epoch: 11 [0/4000982 (0%)]	Loss: 0.47779935598373413086
Train Epoch: 11 [450000/4000982 (11%)]	Loss: 0.46794977784156799316
Train Epoch: 11 [900000/4000982 (22%)]	Loss: 0.46946293115615844727
Train Epoch: 11 [1350000/4000982 (34%)]	Loss: 0.47464880347251892090
Train Epoch: 11 [1800000/4000982 (45%)]	Loss: 0

Train Epoch: 20 [1350000/4000982 (34%)]	Loss: 0.42241734266281127930
Train Epoch: 20 [1800000/4000982 (45%)]	Loss: 0.41251212358474731445
Train Epoch: 20 [2250000/4000982 (56%)]	Loss: 0.41665816307067871094
Train Epoch: 20 [2700000/4000982 (67%)]	Loss: 0.41647925972938537598
Train Epoch: 20 [3150000/4000982 (79%)]	Loss: 0.41939139366149902344
Train Epoch: 20 [3600000/4000982 (90%)]	Loss: 0.42310807108879089355

Test set: Average loss: 0.00002485974619048648, Accuracy: 3439746/4000982 (85.97254374051171055271%)


Test set: Average loss: 0.00002690447036002297, Accuracy: 846549/999996 (84.65523862095447782394%)

Train Epoch: 21 [0/4000982 (0%)]	Loss: 0.40258160233497619629
Train Epoch: 21 [450000/4000982 (11%)]	Loss: 0.40567937493324279785
Train Epoch: 21 [900000/4000982 (22%)]	Loss: 0.41109478473663330078
Train Epoch: 21 [1350000/4000982 (34%)]	Loss: 0.41852617263793945312
Train Epoch: 21 [1800000/4000982 (45%)]	Loss: 0.40680128335952758789
Train Epoch: 21 [2250000/4000982 (56%)]	Loss: 

Train Epoch: 30 [1800000/4000982 (45%)]	Loss: 0.38065031170845031738
Train Epoch: 30 [2250000/4000982 (56%)]	Loss: 0.38200384378433227539
Train Epoch: 30 [2700000/4000982 (67%)]	Loss: 0.38133895397186279297
Train Epoch: 30 [3150000/4000982 (79%)]	Loss: 0.36968111991882324219
Train Epoch: 30 [3600000/4000982 (90%)]	Loss: 0.37836736440658569336

Test set: Average loss: 0.00002239151581306942, Accuracy: 3509285/4000982 (87.71059204965180811087%)


Test set: Average loss: 0.00002468195634719450, Accuracy: 862207/999996 (86.22104488417953405133%)

Train Epoch: 31 [0/4000982 (0%)]	Loss: 0.37016960978507995605
Train Epoch: 31 [450000/4000982 (11%)]	Loss: 0.37347060441970825195
Train Epoch: 31 [900000/4000982 (22%)]	Loss: 0.37891432642936706543
Train Epoch: 31 [1350000/4000982 (34%)]	Loss: 0.37281683087348937988
Train Epoch: 31 [1800000/4000982 (45%)]	Loss: 0.37603914737701416016
Train Epoch: 31 [2250000/4000982 (56%)]	Loss: 0.37837183475494384766
Train Epoch: 31 [2700000/4000982 (67%)]	Loss: 

Train Epoch: 40 [2250000/4000982 (56%)]	Loss: 0.36828202009201049805
Train Epoch: 40 [2700000/4000982 (67%)]	Loss: 0.36622002720832824707
Train Epoch: 40 [3150000/4000982 (79%)]	Loss: 0.36115163564682006836
Train Epoch: 40 [3600000/4000982 (90%)]	Loss: 0.37135592103004455566

Test set: Average loss: 0.00002192195279349107, Accuracy: 3521395/4000982 (88.01326774276914477468%)


Test set: Average loss: 0.00002427564686513506, Accuracy: 864753/999996 (86.47564590258360794905%)

Train Epoch: 41 [0/4000982 (0%)]	Loss: 0.37621039152145385742
Train Epoch: 41 [450000/4000982 (11%)]	Loss: 0.36876460909843444824
Train Epoch: 41 [900000/4000982 (22%)]	Loss: 0.36637973785400390625
Train Epoch: 41 [1350000/4000982 (34%)]	Loss: 0.36822852492332458496
Train Epoch: 41 [1800000/4000982 (45%)]	Loss: 0.37960028648376464844
Train Epoch: 41 [2250000/4000982 (56%)]	Loss: 0.36069649457931518555
Train Epoch: 41 [2700000/4000982 (67%)]	Loss: 0.37013342976570129395
Train Epoch: 41 [3150000/4000982 (79%)]	Loss: 

Train Epoch: 50 [2700000/4000982 (67%)]	Loss: 0.35678148269653320312
Train Epoch: 50 [3150000/4000982 (79%)]	Loss: 0.35825356841087341309
Train Epoch: 50 [3600000/4000982 (90%)]	Loss: 0.36116877198219299316

Test set: Average loss: 0.00002168060746043921, Accuracy: 3527985/4000982 (88.17797730657123622677%)


Test set: Average loss: 0.00002405924897175282, Accuracy: 866413/999996 (86.64164656658626029184%)

Train Epoch: 51 [0/4000982 (0%)]	Loss: 0.37178739905357360840
Train Epoch: 51 [450000/4000982 (11%)]	Loss: 0.37143209576606750488
Train Epoch: 51 [900000/4000982 (22%)]	Loss: 0.36233064532279968262
Train Epoch: 51 [1350000/4000982 (34%)]	Loss: 0.36829590797424316406
Train Epoch: 51 [1800000/4000982 (45%)]	Loss: 0.36336866021156311035
Train Epoch: 51 [2250000/4000982 (56%)]	Loss: 0.35430815815925598145
Train Epoch: 51 [2700000/4000982 (67%)]	Loss: 0.35464003682136535645
Train Epoch: 51 [3150000/4000982 (79%)]	Loss: 0.36167338490486145020
Train Epoch: 51 [3600000/4000982 (90%)]	Loss: 

Train Epoch: 60 [3150000/4000982 (79%)]	Loss: 0.36062785983085632324
Train Epoch: 60 [3600000/4000982 (90%)]	Loss: 0.36755090951919555664

Test set: Average loss: 0.00002163268982258160, Accuracy: 3528961/4000982 (88.20237131784146811242%)


Test set: Average loss: 0.00002401915298833046, Accuracy: 866551/999996 (86.65544662178648138706%)

Train Epoch: 61 [0/4000982 (0%)]	Loss: 0.36996552348136901855
Train Epoch: 61 [450000/4000982 (11%)]	Loss: 0.36015570163726806641
Train Epoch: 61 [900000/4000982 (22%)]	Loss: 0.36410939693450927734
Train Epoch: 61 [1350000/4000982 (34%)]	Loss: 0.36895108222961425781
Train Epoch: 61 [1800000/4000982 (45%)]	Loss: 0.36213368177413940430
Train Epoch: 61 [2250000/4000982 (56%)]	Loss: 0.36454483866691589355
Train Epoch: 61 [2700000/4000982 (67%)]	Loss: 0.36730989813804626465
Train Epoch: 61 [3150000/4000982 (79%)]	Loss: 0.36969462037086486816
Train Epoch: 61 [3600000/4000982 (90%)]	Loss: 0.35206621885299682617

Test set: Average loss: 0.0000216254247789038

Train Epoch: 70 [3600000/4000982 (90%)]	Loss: 0.36278337240219116211

Test set: Average loss: 0.00002161286101909354, Accuracy: 3529514/4000982 (88.21619292463699935070%)


Test set: Average loss: 0.00002400241464783903, Accuracy: 866691/999996 (86.66944667778670918779%)

Train Epoch: 71 [0/4000982 (0%)]	Loss: 0.35774520039558410645
Train Epoch: 71 [450000/4000982 (11%)]	Loss: 0.35813623666763305664
Train Epoch: 71 [900000/4000982 (22%)]	Loss: 0.36741256713867187500
Train Epoch: 71 [1350000/4000982 (34%)]	Loss: 0.36790567636489868164
Train Epoch: 71 [1800000/4000982 (45%)]	Loss: 0.35411942005157470703
Train Epoch: 71 [2250000/4000982 (56%)]	Loss: 0.35749390721321105957
Train Epoch: 71 [2700000/4000982 (67%)]	Loss: 0.36109647154808044434
Train Epoch: 71 [3150000/4000982 (79%)]	Loss: 0.36449322104454040527
Train Epoch: 71 [3600000/4000982 (90%)]	Loss: 0.35600450634956359863

Test set: Average loss: 0.00002161525662813801, Accuracy: 3529454/4000982 (88.21469329279662474619%)


Test set: A

In [14]:
cnn.Test(net, device, criterion, test_loader)


Test set: Average loss: 0.00002399695767962839, Accuracy: 866792/999996 (86.67954671818687018003%)

